In [2]:
import wandb
import pandas as pd

In [3]:
entity = "evangeorgerex"
project = "fwal"

In [12]:
def fetch_runs(entity, project, sweep_ids):
    api = wandb.Api()
    runs_data = []

    for sweep_id in sweep_ids:
        sweep_runs = api.sweep(f"{entity}/{project}/{sweep_id}").runs
        for run in sweep_runs:
            # Check if the run's state is 'finished'
            if run.state == "finished":
                run_name = run.id
                dataset_name = run.config.get("dataset", "Unknown")
                if dataset_name in ["poly_binarised_decimalised_mod10_synth", "simple_trig_synth", "Unknown"]:
                    continue
                
                if run.config.get("as_MLP_baseline", False):
                    model_name = "MLP_same_capacity"
                else:
                    model_name = run.config.get("model", "Unknown")
                    if model_name == "fwal":
                        model_name = "F-Act"
                        
                        
                    # if run.config.get("hierarchical", False):
                    #     model_name = "F-Act hierarchical"
                    #     if run.config.get("share_mask", False):
                    #         model_name = "F-Act hierarchical shared"
                    #         if run.config.get("sigmoid_loss", False):
                    #             model_name = "F-Act hierarchical shared L_sig"
                    # if sweep_id == "x9s9n05b":
                    #     model_name = "F-Act hierarchical TTI (old)"
                    # if sweep_id in ["9npryg91", "7i2o46ri"]:
                    #     model_name = "F-Act hierarchical TTI"
                run_data = {
                    "model": model_name,  # Assuming model name is stored in 'model_name'
                    "run_name":run_name,
                    "sweep_id": sweep_id,
                    "dataset": dataset_name,  # Assuming dataset name is stored in 'dataset_name'
                    "seed": run.config.get("seed_model_init", None),
                    "bestmodel_train/F1_weighted": run.summary.get("bestmodel_train/F1_weighted", float('nan')),
                    "bestmodel_valid/F1_weighted": run.summary.get("bestmodel_valid/F1_weighted", float('nan')),
                    "bestmodel_test/F1_weighted": run.summary.get("bestmodel_test/F1_weighted", float('nan')),
                }
                
                
                # Specific hyperparameters
                if model_name == "lasso":
                    run_data["lasso_C"] = run.config.get("lasso_C")
                    run_data["lasso_l1_ratio"] = run.config.get("lasso_l1_ratio")
                elif model_name == "rf":
                    run_data["rf_max_depth"] = run.config.get("rf_max_depth")
                elif model_name in ["cae", "supervised_cae"]:
                    run_data["CAE_neurons_ratio"] = run.config.get("CAE_neurons_ratio")
                elif model_name == "F-Act hierarchical TTI":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "F-Act hierarchical TTI (old)":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "F-Act":
                    if sweep_id in ["bga4xu49", "4p4opie9"]:
                        run_data['dropout'] = 0.2
                        run_data["lr"] = run.config.get("lr")
                        run_data["num_hidden"] = run.config.get("num_hidden")
                        run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                    else:
                        # sweep_id == "7i2o46ri":
                        run_data['dropout'] = 0.0
                        run_data["lr"] = run.config.get("lr")
                        run_data["num_hidden"] = 4
                        run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "xgboost":
                    run_data["xgb_eta"] = run.config.get("xgb_eta")
                    run_data["xgb_max_depth"] = run.config.get("xgb_max_depth")
                elif model_name in ["MLP_same_capacity", 'SEFS']:
                    run_data["lr"] = run.config.get("lr")
                    run_data["num_hidden"] = run.config.get("num_hidden")
                    
                    
                runs_data.append(run_data)

    return runs_data


def get_best_runs_new(runs_data, hyperparams):
    df = pd.DataFrame(runs_data)

    # Step 1: Group by 'dataset', hyperparameters, and 'seed'. Select the row with the maximum 'bestmodel_valid/F1_weighted' score for each seed.
    max_per_seed = df.groupby(['dataset', 'seed'] + hyperparams).agg({
        'bestmodel_train/F1_weighted': 'max',
        'bestmodel_valid/F1_weighted': 'max',
        'bestmodel_test/F1_weighted': 'max'
    }).reset_index()

    # Step 2: Group by 'dataset' and hyperparameters again, and calculate the mean of the scores across seeds.
    mean_across_seeds = max_per_seed.groupby(['dataset'] + hyperparams).agg({
        'bestmodel_train/F1_weighted': 'mean',
        'bestmodel_valid/F1_weighted': 'mean',
        'bestmodel_test/F1_weighted': 'mean'
    }).reset_index()

    # Step 3: For each 'dataset', find the hyperparameters that lead to the highest average 'bestmodel_valid/F1_weighted'
    optimal_params = mean_across_seeds.loc[mean_across_seeds.groupby('dataset')['bestmodel_valid/F1_weighted'].idxmax()]

    # Step 4: Merge the optimal parameters back to the original dataframe to get the corresponding 'run_name's
    best_runs = pd.merge(
        optimal_params[['dataset'] + hyperparams],
        df,
        on=['dataset'] + hyperparams,
        how='inner'
    ).drop_duplicates(subset=['dataset', 'seed'] + hyperparams)

    # Select only the relevant columns, ensuring to include 'run_name' and other relevant metrics.
    best_run_names = best_runs[['dataset', 'run_name', 'seed', 'sweep_id'] + hyperparams + ['bestmodel_valid/F1_weighted']]
    return best_run_names

def get_best_runs(runs_data, hyperparams):

    df = pd.DataFrame(runs_data)

    # Step 1: Group by 'dataset', 'lasso_C', 'lasso_l1_ratio', and calculate the mean of the F1 scores across seeds.
    grouped = df.groupby(['dataset'] + hyperparams).agg({
        'bestmodel_train/F1_weighted': 'mean',
        'bestmodel_valid/F1_weighted': 'mean',
        'bestmodel_test/F1_weighted': 'mean'
    }).reset_index()

    # Step 2: For each 'dataset', find the hyperparameters that lead to the highest average 'bestmodel_valid/F1_weighted'
    optimal_params = grouped.loc[grouped.groupby('dataset')['bestmodel_valid/F1_weighted'].idxmax()]

    # Print optimal hyperparameters for each dataset
    # print(optimal_params[['dataset', 'lasso_C', 'lasso_l1_ratio', 'bestmodel_valid/F1_weighted']])

    # Step 3: Merge the optimal parameters back to the original dataframe to get the corresponding 'run_name's
    best_runs = pd.merge(
        optimal_params[['dataset'] + hyperparams],
        df,
        on=['dataset'] + hyperparams,
        how='inner'
    )

    # Select only the relevant columns
    best_run_names = best_runs[['dataset', 'run_name', 'seed', 'sweep_id'] + hyperparams +['bestmodel_valid/F1_weighted']]
    return best_run_names

def print_run_ids(best_runs):
    output = ''.join(['- '+rn+'\n' for rn in best_runs['run_name'].to_list()])
    print(output)
    


# Lasso

In [5]:
lasso_sweep_ids = [
    'rdvu3fg8', # everything except finance
    'xpxe64ee', # finance
]

lasso_hyperparameters = ['lasso_C', 'lasso_l1_ratio']

In [6]:
runs_data = fetch_runs(entity, project, lasso_sweep_ids)
best_run_names_lasso = get_best_runs(runs_data, lasso_hyperparameters)


In [9]:
best_run_names_lasso

,dataset,run_name,seed,sweep_id,lasso_C,lasso_l1_ratio,bestmodel_valid/F1_weighted
0,COIL20,2j5c9lxu,2,rdvu3fg8,10,0.50,0.982551
1,COIL20,k88mir2w,1,rdvu3fg8,10,0.50,0.982551
2,COIL20,dwxmsim1,0,rdvu3fg8,10,0.50,0.982551
3,Isolet,gt7tyeln,2,rdvu3fg8,100,0.25,0.947942
4,Isolet,70kvvbze,1,rdvu3fg8,100,0.25,0.947942
5,Isolet,2ptd0x5l,0,rdvu3fg8,100,0.25,0.947942
6,PBMC,wns0vfsv,2,rdvu3fg8,10,0.00,0.873614
7,PBMC,pf3zov6p,1,rdvu3fg8,10,0.00,0.873614
8,PBMC,tfcohywp,0,rdvu3fg8,10,0.00,0.873614
9,USPS,sg718c2e,2,rdvu3fg8,10,0.75,0.934612


In [13]:
print_run_ids(best_run_names_lasso)


- 2j5c9lxu
- k88mir2w
- dwxmsim1
- gt7tyeln
- 70kvvbze
- 2ptd0x5l
- wns0vfsv
- pf3zov6p
- tfcohywp
- sg718c2e
- xceejtd2
- ol6n5ujj
- vf369m2r
- qj04zf76
- rvzko54o
- wfz51c2g
- u8ir7re8
- umceu31l
- bkbz6gaj
- yugvcdf8
- uso6laas
- h0oo4eyt
- 90w4sgar
- ls7yxzfh



# XGBoost

In [8]:
xgb_sweep_ids = [
    'yr11wqmg', # everything
]

xgb_hyperparameters = ['xgb_eta', 'xgb_max_depth']

In [14]:
runs_data_xgb = fetch_runs(entity, project, xgb_sweep_ids)
best_run_names_xgb = get_best_runs(runs_data_xgb, xgb_hyperparameters)


In [15]:
best_run_names_xgb

,dataset,run_name,seed,sweep_id,xgb_eta,xgb_max_depth,bestmodel_valid/F1_weighted
0,COIL20,xgzcuu37,2,yr11wqmg,0.1,6,0.985390
1,COIL20,dnsshn1b,1,yr11wqmg,0.1,6,0.985390
2,COIL20,o88hpkso,0,yr11wqmg,0.1,6,0.985390
3,Isolet,wai78tkf,2,yr11wqmg,0.1,9,0.902887
4,Isolet,mx3xpyr0,1,yr11wqmg,0.1,9,0.902887
5,Isolet,hn9uwhoq,0,yr11wqmg,0.1,9,0.902887
6,PBMC,j3q3ue8i,2,yr11wqmg,0.3,3,0.898703
7,PBMC,6hpzi11o,1,yr11wqmg,0.3,3,0.898703
8,PBMC,qjkk5v8v,0,yr11wqmg,0.3,3,0.898703
9,USPS,0tni6ygc,2,yr11wqmg,0.3,3,0.968251


In [16]:
print_run_ids(best_run_names_xgb)

- xgzcuu37
- dnsshn1b
- o88hpkso
- wai78tkf
- mx3xpyr0
- hn9uwhoq
- j3q3ue8i
- 6hpzi11o
- qjkk5v8v
- 0tni6ygc
- e8b50yxl
- 9xvjwfgt
- knse2dro
- xtun7bzd
- ou92939j
- jl2vveq2
- rzg0omcq
- ld6x4ag6
- 92d7rxam
- zsf4ypb2
- qkznq7x3
- lbp74wl7
- n19mv06e
- 1mijbx36



# MLP

In [17]:
MLP_sweep_ids = [
    's5m2sqsv', # everything
]

MLP_hyperparameters = ['lr', 'num_hidden']

In [19]:
runs_data_MLP = fetch_runs(entity, project, MLP_sweep_ids)
best_run_names_MLP = get_best_runs(runs_data_MLP, MLP_hyperparameters)

In [20]:
best_run_names_MLP

,dataset,run_name,seed,sweep_id,lr,num_hidden,bestmodel_valid/F1_weighted
0,COIL20,srbvb01a,2,s5m2sqsv,0.0010,1,0.993047
1,COIL20,oekx63p8,1,s5m2sqsv,0.0010,1,0.992923
2,COIL20,alprfdqq,0,s5m2sqsv,0.0010,1,0.993025
3,Isolet,d8w94wnf,2,s5m2sqsv,0.0010,2,0.941667
4,Isolet,76lre0vu,1,s5m2sqsv,0.0010,2,0.938385
5,Isolet,4z1wjp9y,0,s5m2sqsv,0.0010,2,0.954844
6,PBMC,kyc5dopk,2,s5m2sqsv,0.0010,4,0.894113
7,PBMC,x3nb16bc,1,s5m2sqsv,0.0010,4,0.903739
8,PBMC,c9xudzv0,0,s5m2sqsv,0.0010,4,0.898891
9,USPS,mp8v0qtq,1,s5m2sqsv,0.0010,1,0.972011


In [21]:
print_run_ids(best_run_names_MLP)


- srbvb01a
- oekx63p8
- alprfdqq
- d8w94wnf
- 76lre0vu
- 4z1wjp9y
- kyc5dopk
- x3nb16bc
- c9xudzv0
- mp8v0qtq
- 7oe60sig
- ccl9a7eu
- rc1n4qi1
- invvzj6h
- 2fqxsay5
- hpty0m0z
- vqc4hdpr
- 3ocbbqie
- nc206z8l
- hii8gyar
- 013sp4et
- j7trhmgk
- l8anh9u4
- sr87k9n5



# F-Act

In [5]:
fact_sweep_ids = [
    'bga4xu49', # everything
    '7i2o46ri', # everything no dropout
    '4p4opie9', # mice_protein and madelon no dropout
    'uw1zvim4', # more PBMC
    'hfqtys2c', # mice_protein and madelon no dropout more patience
]

fact_hyperparameters = ['lr', 'num_hidden', 'sparsity_regularizer_hyperparam', 'dropout']

In [15]:
runs_data_fact = fetch_runs(entity, project, fact_sweep_ids)
best_run_names_fact = get_best_runs(runs_data_fact, fact_hyperparameters)
best_run_names_fact

,dataset,run_name,seed,sweep_id,lr,num_hidden,sparsity_regularizer_hyperparam,dropout,bestmodel_valid/F1_weighted
0,COIL20,bfi5l6qu,2,bga4xu49,0.0010,1,0.0,0.2,0.985809
1,COIL20,ihpfigr4,1,bga4xu49,0.0010,1,0.0,0.2,0.992920
2,COIL20,rhrqy9bb,0,bga4xu49,0.0010,1,0.0,0.2,1.000000
3,Isolet,sj64pl05,2,bga4xu49,0.0003,1,0.1,0.2,0.921679
4,Isolet,dtv0wjox,1,bga4xu49,0.0003,1,0.1,0.2,0.932089
5,Isolet,gp378vp4,0,bga4xu49,0.0003,1,0.1,0.2,0.925571
6,PBMC,vqcdnueq,2,bga4xu49,0.0001,4,10.0,0.2,0.846068
7,USPS,b8nrupvc,2,bga4xu49,0.0010,1,0.1,0.2,0.965020
8,USPS,rcacu3yt,1,bga4xu49,0.0010,1,0.1,0.2,0.971490
9,USPS,swbk9dsz,0,bga4xu49,0.0010,1,0.1,0.2,0.967743


### Runs to report:

- bfi5l6qu
- ihpfigr4
- rhrqy9bb
- sj64pl05
- dtv0wjox
- gp378vp4
- vqcdnueq
- b8nrupvc
- rcacu3yt
- swbk9dsz
- sawnisk5
- dqq6q5ej
- 33iwvq59
- ko5c4p3v
- h4m5vvcm
- zyuohsyk



- 0msrkm0b



- 3v8n9qe6
- 3e4jm286
- n0mcba2f
- 94j8ch8b
- qskac7qn

## Without max filtering:

In [14]:
print_run_ids(best_run_names_fact)

- bfi5l6qu
- ihpfigr4
- rhrqy9bb
- sj64pl05
- dtv0wjox
- gp378vp4
- vqcdnueq
- b8nrupvc
- rcacu3yt
- swbk9dsz
- sawnisk5
- dqq6q5ej
- 33iwvq59
- ko5c4p3v
- h4m5vvcm
- zyuohsyk
- nh4kxwyz
- toim1dzn
- cjaj7kri
- 0msrkm0b
- 9cot8lqv
- ooqtpbvl
- wb2t18o7
- 3v8n9qe6
- 3e4jm286
- n0mcba2f
- 94j8ch8b
- qskac7qn



## With max filtering:

In [6]:
runs_data_fact = fetch_runs(entity, project, fact_sweep_ids)
best_run_names_fact = get_best_runs(runs_data_fact, fact_hyperparameters)

In [7]:
best_run_names_fact

,dataset,run_name,seed,sweep_id,lr,num_hidden,sparsity_regularizer_hyperparam,dropout,bestmodel_valid/F1_weighted
0,COIL20,bfi5l6qu,2,bga4xu49,0.0010,1,0.0,0.2,0.985809
1,COIL20,ihpfigr4,1,bga4xu49,0.0010,1,0.0,0.2,0.992920
2,COIL20,rhrqy9bb,0,bga4xu49,0.0010,1,0.0,0.2,1.000000
3,Isolet,sj64pl05,2,bga4xu49,0.0003,1,0.1,0.2,0.921679
4,Isolet,dtv0wjox,1,bga4xu49,0.0003,1,0.1,0.2,0.932089
5,Isolet,gp378vp4,0,bga4xu49,0.0003,1,0.1,0.2,0.925571
6,PBMC,vqcdnueq,2,bga4xu49,0.0001,4,10.0,0.2,0.846068
7,USPS,b8nrupvc,2,bga4xu49,0.0010,1,0.1,0.2,0.965020
8,USPS,rcacu3yt,1,bga4xu49,0.0010,1,0.1,0.2,0.971490
9,USPS,swbk9dsz,0,bga4xu49,0.0010,1,0.1,0.2,0.967743


In [32]:
best_run_names_fact_to_run = best_run_names_fact[(best_run_names_fact['dataset'] != 'madelon') & (best_run_names_fact['dataset'] != 'mice_protein') ]

In [33]:
print_run_ids(best_run_names_fact_to_run)

- bfi5l6qu
- ihpfigr4
- rhrqy9bb
- sj64pl05
- dtv0wjox
- gp378vp4
- vqcdnueq
- b8nrupvc
- rcacu3yt
- swbk9dsz
- sawnisk5
- dqq6q5ej
- 33iwvq59
- n0mcba2f
- 94j8ch8b
- qskac7qn



In [20]:
print_run_ids(best_run_names_fact)


- bfi5l6qu
- ihpfigr4
- rhrqy9bb
- sj64pl05
- dtv0wjox
- gp378vp4
- vqcdnueq
- b8nrupvc
- rcacu3yt
- swbk9dsz
- sawnisk5
- dqq6q5ej
- 33iwvq59
- ko5c4p3v
- h4m5vvcm
- zyuohsyk
- uua9jral
- 4engzvgx
- 5pb7hj4h
- n0mcba2f
- 94j8ch8b
- qskac7qn



## F-Act identifying PBMC to run
Not all the PBMC runs finished successfully. We need to identify which runs to re-do so that we can get statistical significance.

In [7]:
wumwum = pd.DataFrame(runs_data_fact)

In [8]:
wumwum = wumwum[wumwum['dataset'] == 'PBMC']

In [9]:
grouped = wumwum.groupby(['lr', 'num_hidden', 'sparsity_regularizer_hyperparam'])

# Calculating the mean of 'model_valid/F1_weighted' and counting the number of rows for each group
result = grouped['bestmodel_valid/F1_weighted'].agg(['mean', 'count'])
result

mean  count
lr     num_hidden sparsity_regularizer_hyperparam                 
0.0001 1          0.0                              0.806942      1
                  0.1                              0.829282      2
                  1.0                              0.817140      2
                  10.0                             0.821775      2
                  100.0                            0.807054      2
       2          0.0                              0.829243      2
                  0.1                              0.826875      2
                  0.5                              0.819522      2
                  1.0                              0.836508      1
                  10.0                             0.821743      2
                  100.0                            0.819027      2
       4          0.0                              0.845983      1
                  0.1                              0.845983      1
                  0.5                              0.836357      1
                  1.0                              0.841218      1
                  10.0                             0.846068      1
                  100.0                            0.811643      2
0.0003 1          0.0                              0.826441      1
                  0.1                              0.821525      1
                  0.5                              0.806936      2
                  1.0                              0.811764      2
                  10.0                             0.811757      2
                  100.0                            0.802432      1
       2          0.0                              0.813789      3
                  0.1                              0.803933      3
                  0.5                              0.816825      3
                  1.0                              0.808801      3
                  10.0                             0.813674      3
                  100.0                            0.794588      2
       4          0.0                              0.824909      3
                  0.1                              0.824909      3
                  0.5                              0.824900      3
                  1.0                              0.823263      3
                  10.0                             0.807232      3
                  100.0                            0.800231      3
0.0010 1          0.0                              0.788462      1
                  0.1                              0.802433      1
                  0.5                              0.735485      1
                  1.0                              0.802871      1
                  100.0                            0.778641      2
       2          0.0                              0.744986      1
                  0.1                              0.770242      2
                  0.5                              0.776113      2
                  100.0                            0.797965      1
       4          0.0                              0.764268      2
                  0.1                              0.807336      1
                  1.0                              0.729720      1
                  10.0                             0.805921      3
                  100.0                            0.730106      3

# SEFS

In [26]:
SEFS_sweep_ids = [
    'n6zb5lc9', # everything
]

SEFS_hyperparameters = ['lr', 'num_hidden']

In [27]:
runs_data_SEFS = fetch_runs(entity, project, SEFS_sweep_ids)
best_run_names_SEFS = get_best_runs(runs_data_SEFS, SEFS_hyperparameters)


In [28]:
best_run_names_SEFS

,dataset,run_name,seed,sweep_id,lr,num_hidden,bestmodel_valid/F1_weighted
0,COIL20,4ri15qxr,1,n6zb5lc9,0.003,1,0.961426
1,COIL20,8j0jc7hb,2,n6zb5lc9,0.003,1,0.971665
2,COIL20,n1h3md70,0,n6zb5lc9,0.003,1,0.957945
3,Isolet,5wir5gpp,2,n6zb5lc9,0.003,1,0.900668
4,Isolet,l7x9ro80,1,n6zb5lc9,0.003,1,0.867696
5,Isolet,cie5vyxw,0,n6zb5lc9,0.003,1,0.878427
6,USPS,7r7np8yl,2,n6zb5lc9,0.003,1,0.936241
7,USPS,p8m4bjsz,1,n6zb5lc9,0.003,1,0.933000
8,USPS,m30x375m,0,n6zb5lc9,0.003,1,0.937953
9,finance,q23bgu6e,2,n6zb5lc9,0.003,2,0.565059


In [29]:
print_run_ids(best_run_names_SEFS)


- 4ri15qxr
- 8j0jc7hb
- n1h3md70
- 5wir5gpp
- l7x9ro80
- cie5vyxw
- 7r7np8yl
- p8m4bjsz
- m30x375m
- q23bgu6e
- oomxlr29
- a3oi8xgt
- nii841im
- h9kiqdnu
- y5o7lz1z
- lo2blgf6
- 77wuap81
- cuvpz3wf
- 6rd3gkh9
- gjwugdf5
- zbey3upy

